In [1]:
import pandas as pd
import numpy as np
import math
from math import cos, sin, pi


df_T = pd.read_csv('data/T_amb.csv', header=None)       # 辐射体温度/环境温度
df_e = pd.read_csv('data/cooler.csv', header=None)        # 辐射体发射率
df_a = pd.read_csv('data/cooler.csv', header=None)        # 辐射体吸收率
df_cc = pd.read_csv('data/cc.csv', header=None)        # 云层覆盖率
df_trans = pd.read_csv('data/trans.csv', header=None)        # 大气透过率
T_C = 0.9   # PE膜透过率

df_e.head()

,0,1
0,0.300000,0.893202
1,0.303857,0.867307
2,0.303867,0.851418
3,0.303876,0.837516
4,0.305241,0.807718


In [4]:
# 全天各时刻的辐射功率
# 全天总功

def n_I_BB(W:float,T:float)->float:
    '''
    计算黑体光谱辐射力
    W:辐射体波长
    T:辐射体温度
    '''
    C_1 = 3.743e8
    C_2 = 1.439e4
    return (C_1 * W ** (-5) ) / (np.exp(C_2 / (W * T)) -1)

def P_rad(T1:float)->float:
    '''
    计算辐射功率
    T1:辐射体温度
    调用n_I_BB函数通过辐射体波长和温度计算黑体光谱辐射力，再乘以辐射体发射率、波长差与透过率，最后求和
    输出单个时间点的全波段辐射功率
    '''
    np_df_e = np.array(df_e)
    W = np.array(df_e.iloc[:-1,0]).reshape(1319,)      #辐射体发射率对应的波长
    T = np.full((1319,),T1)       # 辐射体温度
    t_c = np.full((1319,),T_C)     # PE膜透过率0.9
    delta_W = np_df_e[1:,0] - np_df_e[:-1,0]    # 前后项波长差
    np_cooler = np.array(df_e.iloc[:-1,1])        # 辐射体发射率
    rad = np.apply_along_axis(n_I_BB,0,W,T) * np_cooler * delta_W * t_c # 计算辐射力*辐射体发射率*波长差*PE膜透过率 需要保持维度一致
    return rad.sum()

def E_rad(delta_t:int = 60, is_sum:bool = False) -> list :
    '''
    is_sum = True 返回总辐射功率
    is_sum = False 返回每个时间点的辐射功率
    delta_t = 60 为默认时间间隔(s),当delta_t = 1时,返回瞬时辐射功率
    '''
    P_list = []
    np_df_T = np.array(df_T)
    for i in range(len(np_df_T[:,1])):
        if np_df_T[i,1] < np_df_T[i,2]:            # 比较辐射体温度与环境温度大小
            P_list.append(P_rad(np_df_T[i,1]))        # 若辐射体温度＜环境温度，则计算制冷功率
        else:
            P_list.append(0)                          # 若辐射体温度＞环境温度，则制冷功率为0

    # 进行向量化计算
    P_list = np.array(P_list)
    delta_t = np.full((len(P_list),), delta_t)
    P_list = P_list * delta_t
    
    if is_sum:
        return [sum(P_list)]
    else:
        return P_list

print(pd.DataFrame(data = E_rad(delta_t = 60,is_sum=True), columns = ['P_rad day']))
print("\n")
print(pd.DataFrame(data = E_rad(delta_t = 1,is_sum=False), columns = ['P_rad minute']))

      P_rad day
0  1.356367e+07


     P_rad minute
0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
..            ...
895    265.435880
896    266.002493
897    266.454606
898    267.023449
899    267.494181

[900 rows x 1 columns]


In [71]:
# 辐射体吸收的大气辐射功率
DELTA_THETA = 5
CC = 0.15  # 云层覆盖率

def P_atm_font(theta:float = 90,is_summer:bool=True):
    alpha = np.array(df_a.iloc[:-1,1])
    try:
        if is_summer:
            t_atm = np.array(df_trans.iloc[:-1,1])
        else:
            t_atm = np.array(df_trans.iloc[:-1,2])
        args = np.full(t_atm.shape, fill_value=1/np.cos(theta/180*np.pi))
        cc = np.full(t_atm.shape, fill_value=CC)
        e_atm = 1-(t_atm**args)
        res_sum = alpha*e_atm*(1-cc)
        res = pd.DataFrame(data = [alpha,e_atm,1-cc,res_sum]).T
        res.columns = ['alpha','e_atm','1-cc','res_num']
        return res
    except:
        print ("e_atm 函数参数错误")


In [72]:
P_atm_font(theta=90,is_summer=True)

,alpha,e_atm,1-cc,res_num
0,0.893202,1.0,0.85,0.759222
1,0.867307,1.0,0.85,0.737211
2,0.851418,1.0,0.85,0.723706
3,0.837516,1.0,0.85,0.711889
4,0.807718,1.0,0.85,0.686561
...,...,...,...,...
1314,0.838510,1.0,0.85,0.712733
1315,0.967550,1.0,0.85,0.822418
1316,0.856310,1.0,0.85,0.727863
1317,0.745150,1.0,0.85,0.633377


In [52]:
alpha = np.array(df_a.iloc[:-1,1])
t_atm = np.array(df_trans.iloc[:-1,2])
args = np.full(t_atm.shape, fill_value=1/np.cos(5/180*np.pi))
res = alpha*(1-(t_atm**args))

In [61]:
b = pd.DataFrame([alpha,t_atm,args]).T
b.columns = ['alpha','t_atm','args']

In [44]:
np.arange(0,90,DELTA_THETA)

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
       85])

In [3]:
def P_atm(T1):
    np_df_a = np.array(df_a)
    np_df_T = np.array(df_T)
    np_df_t_atm = np.array(df_trans)
    np_df_cc = np.array(df_cc)
    W = np.array(df_e.iloc[:-1,0]).reshape(1319,)      # 辐射体发射率对应的波长
    T = np.full((1319,),T1)       # 辐射体温度
    t_c = np.full((1319,),0.9)     # PE膜透过率0.9
    e_atm = 1 - np.array(df_trans.iloc[:-1,1]).reshape(1319,)
    e_cloud = np.full((1319,),0.75)     # 云层发射率0.75
    j = np.arange(0,18,1)
    angle_cos = np.cos(j * pi / 18 / 2)
    angle_sin = np.sin(j * pi / 18 / 2)
    delta_angle = 5/180 * np.pi
    delta_W = np_df_a[1:,0] - np_df_a[:-1,0]
    np_cooler = np.array(df_a.iloc[:-1,1])        # 辐射体吸收率
    np_cc = np.array(df_cc.iloc[:-1,1])
    atm = np.apply_along_axis(n_I_BB,0,W,T) * np_cooler * (e_atm * (1 - np_cc) + e_cloud * np_cc) * angle_cos * angle_sin * delta_angle * delta_W * t_c * 2
    return atm.sum()

def E_atm(delta_t = 60, is_sum:bool = False) -> list :
    '''
    is_sum = True 返回总辐射功率
    is_sum = False 返回每个时间点的辐射功率
    delta_t = 60 为默认时间间隔(s),当delta_t = 1时,返回瞬时辐射功率
    '''
    P_list = []
    np_df_e = np.array(df_e)
    np_df_T = np.array(df_T)
    for i in range(len(np_df_T[:,1])):
        if np_df_T[i,1] < np_df_T[i,2]:
            P_list.append(P_atm(np_df_T[i,2]))
        else:
            P_list.append(0)

    # 进行向量化计算
    P_list = np.array(P_list)
    delta_t = np.full((len(P_list),), delta_t)
    P_list = P_list * delta_t
    
    if is_sum:
        return [sum(P_list)]
    else:
        return P_list

print(pd.DataFrame(data = E_atm(delta_t = 60,is_sum=True), columns = ['P_atm day']))
print("\n")
print(pd.DataFrame(data = E_atm(delta_t = 1,is_sum=False), columns = ['P_atm minute']))

ValueError: operands could not be broadcast together with shapes (1319,) (899,) 